In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import RidgeCV
from sktime.regression.kernel_based import RocketRegressor
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from keras.layers import LSTM, Dense

In [116]:
from sklearn.neighbors import KNeighborsRegressor

In [45]:
from sktime.regression.deep_learning import CNNRegressor

In [4]:
import os

def list_files_recursive(directory):
    file_list = []  # 파일 이름을 저장할 빈 리스트를 생성
    for root, dirs, files in os.walk(directory):
        for name in files:
            file_list.append(os.path.join(root, name))  # 리스트에 파일 이름 추가
    return file_list  # 리스트 반환

# 사용 예:
directory = "./Test" 
test_col = list_files_recursive(directory)

In [3]:
test_col

['./Test\\114_Autumn.csv',
 './Test\\114_Spring.csv',
 './Test\\114_Summer.csv',
 './Test\\114_Winter.csv',
 './Test\\136_Autumn.csv',
 './Test\\136_Spring.csv',
 './Test\\136_Summer.csv',
 './Test\\136_Winter.csv',
 './Test\\146_Autumn.csv',
 './Test\\146_Spring.csv',
 './Test\\146_Summer.csv',
 './Test\\146_Winter.csv',
 './Test\\177_Autumn.csv',
 './Test\\177_Spring.csv',
 './Test\\177_Summer.csv',
 './Test\\177_Winter.csv',
 './Test\\189_Autumn.csv',
 './Test\\189_Spring.csv',
 './Test\\189_Summer.csv',
 './Test\\189_Winter.csv',
 './Test\\203_Autumn.csv',
 './Test\\203_Spring.csv',
 './Test\\203_Summer.csv',
 './Test\\203_Winter.csv',
 './Test\\232_Autumn.csv',
 './Test\\232_Spring.csv',
 './Test\\232_Summer.csv',
 './Test\\232_Winter.csv',
 './Test\\251_Autumn.csv',
 './Test\\251_Spring.csv',
 './Test\\251_Summer.csv',
 './Test\\251_Winter.csv',
 './Test\\263_Autumn.csv',
 './Test\\263_Spring.csv',
 './Test\\263_Summer.csv',
 './Test\\263_Winter.csv',
 './Test\\276_Autumn.csv',
 

In [4]:
train = pd.read_csv("Train_data.csv")
train['Date'] = pd.to_datetime(train['Date'])
train_spring = train[train['Date'].dt.month.isin([2, 3, 4])]

# LinearRegression

In [11]:
def linear_mae(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = LinearRegression()

            # Fit the model
            model.fit(x_train_df, y_train)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results
    

In [14]:
linear_spring_mae, linear_spring_TC_pred = linear_mae('spring')
linear_summer_mae, linear_summer_TC_pred = linear_mae('summer')
linear_autumn_mae, linear_autumn_TC_pred = linear_mae('autumn')
linear_winter_mae, linear_winter_TC_pred = linear_mae('winter')

In [79]:
# Convert each dictionary to a DataFrame
linear_spring_df = pd.DataFrame.from_dict(linear_spring_mae, orient='index', columns=['MAE'])
linear_summer_df = pd.DataFrame.from_dict(linear_summer_mae, orient='index', columns=['MAE'])
linear_autumn_df = pd.DataFrame.from_dict(linear_autumn_mae, orient='index', columns=['MAE'])
linear_winter_df = pd.DataFrame.from_dict(linear_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
linear_mae_df = pd.concat([linear_spring_df, linear_summer_df, linear_autumn_df, linear_winter_df], axis=0)
linear_mae_df.columns = ['linear_MAE']

In [81]:
linear_mae_df

,linear_MAE
114_Spring,2.743749
136_Spring,3.265195
146_Spring,3.001608
177_Spring,2.367195
189_Spring,2.348595
203_Spring,2.231211
232_Spring,2.306668
251_Spring,2.368921
263_Spring,2.165869
276_Spring,2.223393


# XGBoost

In [17]:
def xgboost_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = XGBRegressor(objective='reg:squarederror', n_estimators=150)

            # Fit the model
            model.fit(x_train_df, y_train)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results



In [18]:
xgb_spring_mae, xbg_spring_TC_pred = xgboost_model('spring')
xgb_summer_mae, xgb_summer_TC_pred = xgboost_model('summer')
xgb_autumn_mae, xgb_autumn_TC_pred = xgboost_model('autumn')
xgb_winter_mae, xgb_winter_TC_pred = xgboost_model('winter')

In [82]:
# Convert each dictionary to a DataFrame
xgb_spring_df = pd.DataFrame.from_dict(xgb_spring_mae, orient='index', columns=['MAE'])
xgb_summer_df = pd.DataFrame.from_dict(xgb_summer_mae, orient='index', columns=['MAE'])
xgb_autumn_df = pd.DataFrame.from_dict(xgb_autumn_mae, orient='index', columns=['MAE'])
xgb_winter_df = pd.DataFrame.from_dict(xgb_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
xgb_mae_df = pd.concat([xgb_spring_df, xgb_summer_df, xgb_autumn_df, xgb_winter_df], axis=0)
xgb_mae_df.columns = ['xgb_MAE']


In [83]:
xgb_mae_df

,xgb_MAE
114_Spring,2.659789
136_Spring,3.348211
146_Spring,3.117938
177_Spring,1.931322
189_Spring,2.304510
203_Spring,2.079570
232_Spring,1.965554
251_Spring,2.051523
263_Spring,2.118831
276_Spring,2.320581


# RandomForest_model

In [21]:
from sklearn.ensemble import RandomForestRegressor

def RandomForest_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model
            model = RandomForestRegressor(n_estimators=100, random_state=42)

            # Fit the model
            model.fit(x_train_df, y_train.ravel())

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [22]:
randomforest_spring_mae, randomforest_spring_TC_pred = RandomForest_model('spring')
randomforest_summer_mae, randomforest_summer_TC_pred = RandomForest_model('summer')
randomforest_autumn_mae, randomforest_autumn_TC_pred = RandomForest_model('autumn')
randomforest_winter_mae, randomforest_winter_TC_pred = RandomForest_model('winter')

In [84]:
# Convert each dictionary to a DataFrame
randomforest_spring_df = pd.DataFrame.from_dict(randomforest_spring_mae, orient='index', columns=['MAE'])
randomforest_summer_df = pd.DataFrame.from_dict(randomforest_summer_mae, orient='index', columns=['MAE'])
randomforest_autumn_df = pd.DataFrame.from_dict(randomforest_autumn_mae, orient='index', columns=['MAE'])
randomforest_winter_df = pd.DataFrame.from_dict(randomforest_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
randomforest_mae_df = pd.concat([randomforest_spring_df, randomforest_summer_df, randomforest_autumn_df, randomforest_winter_df], axis=0)
randomforest_mae_df.columns = ['randomforest_MAE']

In [85]:
randomforest_mae_df

,randomforest_MAE
114_Spring,2.684001
136_Spring,3.375208
146_Spring,3.167067
177_Spring,1.988846
189_Spring,2.381969
203_Spring,2.101277
232_Spring,2.001568
251_Spring,2.096164
263_Spring,1.923695
276_Spring,2.265904


# MLP

In [25]:
from keras.callbacks import EarlyStopping

In [49]:
def MLP_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'Spring':
        train = train[train['Date'].dt.month.isin([3, 4, 5])]
    elif weather == 'Summer':
        train = train[train['Date'].dt.month.isin([6, 7, 8])]
    elif weather == 'Autumn':
        train = train[train['Date'].dt.month.isin([9, 10, 11])]
    elif weather == 'Winter':
        train = train[train['Date'].dt.month.isin([12, 1, 2])]

    scaler = StandardScaler()
    y_scaler = StandardScaler()

    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create the model structure
            model = Sequential()
            model.add(Dense(64, input_dim=x_train_df.shape[1], activation='relu'))  # Input layer
            model.add(Dense(32, activation='relu'))  # Hidden layer
            model.add(Dense(1))  # Output layer

            # Compile the model
            model.compile(loss='mean_absolute_error', optimizer='adam')

            # Create early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=10)

            # Fit the model
            model.fit(x_train_df, y_train, epochs=50, batch_size=64, verbose=2, callbacks=[early_stopping], validation_split=0.2)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred)

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results


In [51]:
MLP_spring_mae, MLP_spring_TC_pred = MLP_model('spring')
MLP_summer_mae, MLP_summer_TC_pred = MLP_model('summer')
MLP_autumn_mae, MLP_autumn_TC_pred = MLP_model('autumn')
MLP_winter_mae, MLP_winter_TC_pred = MLP_model('winter')

Epoch 1/50
5478/5478 - 4s - loss: 0.1827 - val_loss: 0.1617 - 4s/epoch - 705us/step
Epoch 2/50
5478/5478 - 3s - loss: 0.1741 - val_loss: 0.1626 - 3s/epoch - 634us/step
Epoch 3/50
5478/5478 - 3s - loss: 0.1722 - val_loss: 0.1600 - 3s/epoch - 632us/step
Epoch 4/50
5478/5478 - 3s - loss: 0.1712 - val_loss: 0.1632 - 3s/epoch - 621us/step
Epoch 5/50
5478/5478 - 4s - loss: 0.1705 - val_loss: 0.1590 - 4s/epoch - 646us/step
Epoch 6/50
5478/5478 - 3s - loss: 0.1699 - val_loss: 0.1624 - 3s/epoch - 637us/step
Epoch 7/50
5478/5478 - 3s - loss: 0.1696 - val_loss: 0.1638 - 3s/epoch - 623us/step
Epoch 8/50
5478/5478 - 3s - loss: 0.1693 - val_loss: 0.1625 - 3s/epoch - 632us/step
Epoch 9/50
5478/5478 - 3s - loss: 0.1688 - val_loss: 0.1587 - 3s/epoch - 638us/step
Epoch 10/50
5478/5478 - 3s - loss: 0.1687 - val_loss: 0.1659 - 3s/epoch - 636us/step
Epoch 11/50
5478/5478 - 3s - loss: 0.1684 - val_loss: 0.1648 - 3s/epoch - 616us/step
Epoch 12/50
5478/5478 - 3s - loss: 0.1682 - val_loss: 0.1624 - 3s/epoch - 

Epoch 39/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1612 - 3s/epoch - 618us/step
Epoch 40/50
5478/5478 - 3s - loss: 0.1655 - val_loss: 0.1593 - 3s/epoch - 618us/step
Epoch 41/50
5478/5478 - 3s - loss: 0.1655 - val_loss: 0.1650 - 3s/epoch - 613us/step
Epoch 42/50
5478/5478 - 3s - loss: 0.1654 - val_loss: 0.1600 - 3s/epoch - 617us/step
Epoch 43/50
5478/5478 - 4s - loss: 0.1654 - val_loss: 0.1591 - 4s/epoch - 651us/step
Epoch 44/50
5478/5478 - 4s - loss: 0.1653 - val_loss: 0.1641 - 4s/epoch - 643us/step
Epoch 45/50
5478/5478 - 3s - loss: 0.1653 - val_loss: 0.1592 - 3s/epoch - 636us/step
Epoch 46/50
5478/5478 - 3s - loss: 0.1653 - val_loss: 0.1608 - 3s/epoch - 637us/step
Epoch 47/50
5478/5478 - 3s - loss: 0.1653 - val_loss: 0.1584 - 3s/epoch - 635us/step
Epoch 48/50
5478/5478 - 3s - loss: 0.1652 - val_loss: 0.1595 - 3s/epoch - 614us/step
Epoch 49/50
5478/5478 - 3s - loss: 0.1652 - val_loss: 0.1572 - 3s/epoch - 612us/step
Epoch 50/50
5478/5478 - 3s - loss: 0.1652 - val_loss: 0.1580 - 3s

5478/5478 - 3s - loss: 0.1662 - val_loss: 0.1615 - 3s/epoch - 608us/step
Epoch 27/50
5478/5478 - 3s - loss: 0.1663 - val_loss: 0.1579 - 3s/epoch - 608us/step
Epoch 28/50
5478/5478 - 3s - loss: 0.1661 - val_loss: 0.1605 - 3s/epoch - 605us/step
Epoch 29/50
5478/5478 - 3s - loss: 0.1660 - val_loss: 0.1641 - 3s/epoch - 608us/step
Epoch 30/50
5478/5478 - 3s - loss: 0.1660 - val_loss: 0.1596 - 3s/epoch - 610us/step
Epoch 31/50
5478/5478 - 3s - loss: 0.1659 - val_loss: 0.1606 - 3s/epoch - 611us/step
Epoch 32/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1598 - 3s/epoch - 617us/step
Epoch 33/50
5478/5478 - 3s - loss: 0.1657 - val_loss: 0.1627 - 3s/epoch - 611us/step
Epoch 34/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1601 - 3s/epoch - 614us/step
Epoch 35/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1643 - 3s/epoch - 607us/step
Epoch 36/50
5478/5478 - 3s - loss: 0.1655 - val_loss: 0.1613 - 3s/epoch - 612us/step
Epoch 37/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1609 - 3s/epoch - 606

5478/5478 - 3s - loss: 0.1680 - val_loss: 0.1651 - 3s/epoch - 610us/step
Epoch 15/50
5478/5478 - 3s - loss: 0.1679 - val_loss: 0.1612 - 3s/epoch - 607us/step
Epoch 16/50
5478/5478 - 3s - loss: 0.1678 - val_loss: 0.1714 - 3s/epoch - 609us/step
Epoch 17/50
5478/5478 - 3s - loss: 0.1677 - val_loss: 0.1648 - 3s/epoch - 598us/step
Epoch 18/50
5478/5478 - 3s - loss: 0.1676 - val_loss: 0.1613 - 3s/epoch - 607us/step
Epoch 19/50
5478/5478 - 3s - loss: 0.1674 - val_loss: 0.1619 - 3s/epoch - 607us/step
Epoch 20/50
5478/5478 - 3s - loss: 0.1674 - val_loss: 0.1636 - 3s/epoch - 602us/step
Epoch 21/50
5478/5478 - 3s - loss: 0.1672 - val_loss: 0.1598 - 3s/epoch - 600us/step
Epoch 22/50
5478/5478 - 3s - loss: 0.1671 - val_loss: 0.1596 - 3s/epoch - 616us/step
Epoch 23/50
5478/5478 - 3s - loss: 0.1670 - val_loss: 0.1623 - 3s/epoch - 609us/step
Epoch 24/50
5478/5478 - 3s - loss: 0.1669 - val_loss: 0.1606 - 3s/epoch - 610us/step
Epoch 25/50
5478/5478 - 3s - loss: 0.1668 - val_loss: 0.1619 - 3s/epoch - 634

5478/5478 - 3s - loss: 0.1744 - val_loss: 0.1613 - 3s/epoch - 607us/step
Epoch 3/50
5478/5478 - 3s - loss: 0.1728 - val_loss: 0.1591 - 3s/epoch - 609us/step
Epoch 4/50
5478/5478 - 3s - loss: 0.1719 - val_loss: 0.1628 - 3s/epoch - 602us/step
Epoch 5/50
5478/5478 - 3s - loss: 0.1712 - val_loss: 0.1589 - 3s/epoch - 606us/step
Epoch 6/50
5478/5478 - 3s - loss: 0.1707 - val_loss: 0.1620 - 3s/epoch - 605us/step
Epoch 7/50
5478/5478 - 3s - loss: 0.1702 - val_loss: 0.1629 - 3s/epoch - 607us/step
Epoch 8/50
5478/5478 - 3s - loss: 0.1700 - val_loss: 0.1641 - 3s/epoch - 607us/step
Epoch 9/50
5478/5478 - 3s - loss: 0.1696 - val_loss: 0.1598 - 3s/epoch - 606us/step
Epoch 10/50
5478/5478 - 3s - loss: 0.1693 - val_loss: 0.1637 - 3s/epoch - 610us/step
Epoch 11/50
5478/5478 - 3s - loss: 0.1690 - val_loss: 0.1640 - 3s/epoch - 605us/step
Epoch 12/50
5478/5478 - 3s - loss: 0.1688 - val_loss: 0.1620 - 3s/epoch - 608us/step
Epoch 13/50
5478/5478 - 3s - loss: 0.1686 - val_loss: 0.1629 - 3s/epoch - 605us/step

...

5478/5478 - 3s - loss: 0.1666 - val_loss: 0.1630 - 3s/epoch - 613us/step
Epoch 25/50
5478/5478 - 3s - loss: 0.1666 - val_loss: 0.1641 - 3s/epoch - 617us/step
Epoch 26/50
5478/5478 - 3s - loss: 0.1665 - val_loss: 0.1614 - 3s/epoch - 599us/step
Epoch 27/50
5478/5478 - 3s - loss: 0.1665 - val_loss: 0.1581 - 3s/epoch - 604us/step
Epoch 28/50
5478/5478 - 3s - loss: 0.1663 - val_loss: 0.1590 - 3s/epoch - 613us/step
Epoch 29/50
5478/5478 - 3s - loss: 0.1663 - val_loss: 0.1608 - 3s/epoch - 615us/step
Epoch 30/50
5478/5478 - 3s - loss: 0.1662 - val_loss: 0.1588 - 3s/epoch - 598us/step
Epoch 31/50
5478/5478 - 3s - loss: 0.1661 - val_loss: 0.1634 - 3s/epoch - 623us/step
Epoch 32/50
5478/5478 - 3s - loss: 0.1660 - val_loss: 0.1591 - 3s/epoch - 615us/step
Epoch 33/50
5478/5478 - 3s - loss: 0.1660 - val_loss: 0.1604 - 3s/epoch - 614us/step
Epoch 34/50
5478/5478 - 3s - loss: 0.1659 - val_loss: 0.1604 - 3s/epoch - 621us/step
Epoch 35/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1666 - 3s/epoch - 613

5478/5478 - 3s - loss: 0.1681 - val_loss: 0.1623 - 3s/epoch - 616us/step
Epoch 13/50
5478/5478 - 3s - loss: 0.1679 - val_loss: 0.1619 - 3s/epoch - 603us/step
Epoch 14/50
5478/5478 - 3s - loss: 0.1676 - val_loss: 0.1644 - 3s/epoch - 601us/step
Epoch 15/50
5478/5478 - 3s - loss: 0.1675 - val_loss: 0.1610 - 3s/epoch - 615us/step
Epoch 16/50
5478/5478 - 3s - loss: 0.1674 - val_loss: 0.1706 - 3s/epoch - 601us/step
Epoch 17/50
5478/5478 - 3s - loss: 0.1673 - val_loss: 0.1643 - 3s/epoch - 591us/step
Epoch 18/50
5478/5478 - 3s - loss: 0.1673 - val_loss: 0.1614 - 3s/epoch - 591us/step
Epoch 19/50
5478/5478 - 3s - loss: 0.1671 - val_loss: 0.1626 - 3s/epoch - 590us/step
Epoch 20/50
5478/5478 - 3s - loss: 0.1669 - val_loss: 0.1615 - 3s/epoch - 590us/step
Epoch 21/50
5478/5478 - 3s - loss: 0.1669 - val_loss: 0.1613 - 3s/epoch - 580us/step
Epoch 22/50
5478/5478 - 3s - loss: 0.1668 - val_loss: 0.1586 - 3s/epoch - 587us/step
Epoch 23/50
5478/5478 - 3s - loss: 0.1666 - val_loss: 0.1609 - 3s/epoch - 585

5478/5478 - 3s - loss: 0.1654 - val_loss: 0.1589 - 3s/epoch - 611us/step
72/72 [==============================] - 0s 508us/step
Epoch 1/50
5478/5478 - 4s - loss: 0.1807 - val_loss: 0.1643 - 4s/epoch - 681us/step
Epoch 2/50
5478/5478 - 3s - loss: 0.1738 - val_loss: 0.1624 - 3s/epoch - 606us/step
Epoch 3/50
5478/5478 - 3s - loss: 0.1722 - val_loss: 0.1601 - 3s/epoch - 602us/step
Epoch 4/50
5478/5478 - 3s - loss: 0.1712 - val_loss: 0.1622 - 3s/epoch - 596us/step
Epoch 5/50
5478/5478 - 3s - loss: 0.1706 - val_loss: 0.1591 - 3s/epoch - 593us/step
Epoch 6/50
5478/5478 - 3s - loss: 0.1701 - val_loss: 0.1609 - 3s/epoch - 607us/step
Epoch 7/50
5478/5478 - 3s - loss: 0.1698 - val_loss: 0.1619 - 3s/epoch - 613us/step
Epoch 8/50
5478/5478 - 3s - loss: 0.1695 - val_loss: 0.1640 - 3s/epoch - 604us/step
Epoch 9/50
5478/5478 - 3s - loss: 0.1692 - val_loss: 0.1597 - 3s/epoch - 607us/step
Epoch 10/50
5478/5478 - 3s - loss: 0.1689 - val_loss: 0.1648 - 3s/epoch - 598us/step
Epoch 11/50
5478/5478 - 3s - lo

5478/5478 - 3s - loss: 0.1660 - val_loss: 0.1617 - 3s/epoch - 598us/step
Epoch 39/50
5478/5478 - 3s - loss: 0.1659 - val_loss: 0.1614 - 3s/epoch - 596us/step
Epoch 40/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1604 - 3s/epoch - 600us/step
Epoch 41/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1633 - 3s/epoch - 596us/step
Epoch 42/50
5478/5478 - 3s - loss: 0.1657 - val_loss: 0.1612 - 3s/epoch - 605us/step
Epoch 43/50
5478/5478 - 3s - loss: 0.1657 - val_loss: 0.1609 - 3s/epoch - 616us/step
Epoch 44/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1663 - 3s/epoch - 622us/step
Epoch 45/50
5478/5478 - 3s - loss: 0.1656 - val_loss: 0.1610 - 3s/epoch - 605us/step
Epoch 46/50
5478/5478 - 3s - loss: 0.1657 - val_loss: 0.1609 - 3s/epoch - 618us/step
Epoch 47/50
5478/5478 - 3s - loss: 0.1655 - val_loss: 0.1592 - 3s/epoch - 604us/step
Epoch 48/50
5478/5478 - 3s - loss: 0.1654 - val_loss: 0.1603 - 3s/epoch - 611us/step
Epoch 49/50
5478/5478 - 3s - loss: 0.1655 - val_loss: 0.1578 - 3s/epoch - 607

5478/5478 - 3s - loss: 0.1663 - val_loss: 0.1628 - 3s/epoch - 595us/step
Epoch 27/50
5478/5478 - 3s - loss: 0.1663 - val_loss: 0.1591 - 3s/epoch - 588us/step
Epoch 28/50
5478/5478 - 3s - loss: 0.1662 - val_loss: 0.1588 - 3s/epoch - 588us/step
Epoch 29/50
5478/5478 - 3s - loss: 0.1661 - val_loss: 0.1609 - 3s/epoch - 601us/step
Epoch 30/50
5478/5478 - 3s - loss: 0.1661 - val_loss: 0.1586 - 3s/epoch - 595us/step
Epoch 31/50
5478/5478 - 3s - loss: 0.1661 - val_loss: 0.1622 - 3s/epoch - 611us/step
Epoch 32/50
5478/5478 - 3s - loss: 0.1659 - val_loss: 0.1585 - 3s/epoch - 598us/step
Epoch 33/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1610 - 3s/epoch - 616us/step
Epoch 34/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1600 - 3s/epoch - 593us/step
Epoch 35/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1655 - 3s/epoch - 603us/step
Epoch 36/50
5478/5478 - 3s - loss: 0.1657 - val_loss: 0.1615 - 3s/epoch - 601us/step
Epoch 37/50
5478/5478 - 3s - loss: 0.1658 - val_loss: 0.1616 - 3s/epoch - 591

In [86]:
# Convert each dictionary to a DataFrame
MLP_spring_df = pd.DataFrame.from_dict(MLP_spring_mae, orient='index', columns=['MAE'])
MLP_summer_df = pd.DataFrame.from_dict(MLP_summer_mae, orient='index', columns=['MAE'])
MLP_autumn_df = pd.DataFrame.from_dict(MLP_autumn_mae, orient='index', columns=['MAE'])
MLP_winter_df = pd.DataFrame.from_dict(MLP_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
MLP_mae_df = pd.concat([MLP_spring_df, MLP_summer_df, MLP_autumn_df, MLP_winter_df], axis=0)
MLP_mae_df.columns = ['MLP_MAE']

In [87]:
MLP_mae_df

,MLP_MAE
114_Spring,2.670100
136_Spring,3.340331
146_Spring,3.101705
177_Spring,1.965631
189_Spring,2.252428
203_Spring,2.003833
232_Spring,1.956227
251_Spring,1.807717
263_Spring,1.920388
276_Spring,2.225343


# Rocket

In [150]:
def Rocket_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))

    x_train = from_2d_array_to_nested(pd.DataFrame(x_train))
    y_train = y_train.ravel()

    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            
            x_test = from_2d_array_to_nested(pd.DataFrame(x_test))
            y_test = test.iloc[:,-1].values.ravel()

            # Create a model
            reg = RocketRegressor(num_kernels=500)

            # Fit the model
            reg.fit(x_train, y_train)

            # Make predictions
            y_pred = reg.predict(x_test)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results


In [151]:
ROCKET_spring_mae, ROCKET_spring_TC_pred = Rocket_model('spring')
ROCKET_summer_mae, ROCKET_summer_TC_pred = Rocket_model('summer')
ROCKET_autumn_mae, ROCKET_autumn_TC_pred = Rocket_model('autumn')
ROCKET_winter_mae, ROCKET_winter_TC_pred = Rocket_model('winter')

In [152]:
# Convert each dictionary to a DataFrame
ROCKET_spring_df = pd.DataFrame.from_dict(ROCKET_spring_mae, orient='index', columns=['MAE'])
ROCKET_summer_df = pd.DataFrame.from_dict(ROCKET_summer_mae, orient='index', columns=['MAE'])
ROCKET_autumn_df = pd.DataFrame.from_dict(ROCKET_autumn_mae, orient='index', columns=['MAE'])
ROCKET_winter_df = pd.DataFrame.from_dict(ROCKET_winter_mae, orient='index', columns=['MAE'])

# Concatenate all the DataFrames along columns
ROCKET_mae_df = pd.concat([ROCKET_spring_df, ROCKET_summer_df, ROCKET_autumn_df, ROCKET_winter_df], axis=0)
ROCKET_mae_df.columns = ['ROCKET_MAE']


In [153]:
ROCKET_mae_df

,ROCKET_MAE
114_Spring,3.175397
136_Spring,3.824690
146_Spring,3.461678
177_Spring,2.651597
189_Spring,2.713350
203_Spring,2.577076
232_Spring,2.446223
251_Spring,2.353716
263_Spring,2.501096
276_Spring,2.998278


In [154]:
MAE_df = pd.concat([linear_mae_df, xgb_mae_df, randomforest_mae_df, MLP_mae_df, ROCKET_mae_df], axis = 1)

In [155]:
MAE_df

,linear_MAE,xgb_MAE,randomforest_MAE,MLP_MAE,ROCKET_MAE
114_Spring,2.743749,2.659789,2.684001,2.670100,3.175397
136_Spring,3.265195,3.348211,3.375208,3.340331,3.824690
146_Spring,3.001608,3.117938,3.167067,3.101705,3.461678
177_Spring,2.367195,1.931322,1.988846,1.965631,2.651597
189_Spring,2.348595,2.304510,2.381969,2.252428,2.713350
203_Spring,2.231211,2.079570,2.101277,2.003833,2.577076
232_Spring,2.306668,1.965554,2.001568,1.956227,2.446223
251_Spring,2.368921,2.051523,2.096164,1.807717,2.353716
263_Spring,2.165869,2.118831,1.923695,1.920388,2.501096
276_Spring,2.223393,2.320581,2.265904,2.225343,2.998278


In [156]:
MAE_df.to_csv('modeling_mae.csv', encoding = 'utf-8')

# Lightgbm

In [99]:
import lightgbm as lgb

In [131]:
def Lightgbm_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = y_scaler.transform(test.iloc[:,-1].values.reshape(-1, 1))

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # laoding data
            lgb_train = lgb.Dataset(x_train_df, y_train)
            lgb_eval = lgb.Dataset(x_test_df, y_test, reference=lgb_train)

            # fitting the model
            model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)

            # Make predictions
            y_pred = model.predict(x_test_df)
    
            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
            y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1))
        

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results



In [132]:
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

LIGHTGBM_spring_mae, LIGHTGBM_spring_TC_pred = Lightgbm_model('spring')
LIGHTGBM_summer_mae, LIGHTGBM_summer_TC_pred = Lightgbm_model('summer')
LIGHTGBM_autumn_mae, LIGHTGBM_autumn_TC_pred = Lightgbm_model('autumn')
LIGHTGBM_winter_mae, LIGHTGBM_winter_TC_pred = Lightgbm_model('winter')

[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l2: 0.997207	valid_0's l1: 0.816728
[2]	valid_0's l2: 0.85694	valid_0's l1: 0.757764
[3]	valid_0's l2: 0.743612	valid_0's l1: 0.706721
[4]	valid_0's l2: 0.649245	valid_0's l1: 0.661741
[5]	valid_0's l2: 0.574219	valid_0's l1: 0.623156
[6]	valid_0's l2: 0.509322	valid_0's l1: 0.588822
[7]	valid_0's l2: 0.458096	valid_0's l1: 0.559908
[8]	valid_0's l2: 0.414544	valid_0's l1: 0.534652
[9]	valid_0's l2: 0.379581	valid_0's l1: 0.512896
[10]	valid_0's l2: 0.350052	valid_0's l1: 0.493631
[11]	valid_0's l2: 0.324974	valid_0's l1: 0.476588
[12]	valid_0's l2: 0.304344	valid_0's l1: 0.461144
[13]	valid_0's l2: 0.287704	valid_0's l1: 0.447886
[14]	valid_0's l2: 0.271329	valid_0's l1: 0.43442
[15]	valid_0's l2: 0.259779	valid_0's l1: 0.423992
[16]	valid_0's l2: 0.249211	valid_0's l1: 0.414133
[17]	valid_0's l2: 0.242273	valid_0's l1: 0.406923
[18]	valid_0's l2: 0.233807	valid_0's l1: 0.397886
[19]	valid_0's l2: 0.226785	valid_0's l

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[58]	valid_0's l2: 0.285984	valid_0's l1: 0.37992
[59]	valid_0's l2: 0.285373	valid_0's l1: 0.379506
[60]	valid_0's l2: 0.285437	valid_0's l1: 0.379171
[61]	valid_0's l2: 0.28497	valid_0's l1: 0.378331
[62]	valid_0's l2: 0.284815	valid_0's l1: 0.377849
[63]	valid_0's l2: 0.284971	valid_0's l1: 0.377826
[64]	valid_0's l2: 0.284914	valid_0's l1: 0.377489
[65]	valid_0's l2: 0.284152	valid_0's l1: 0.376609
[66]	valid_0's l2: 0.282781	valid_0's l1: 0.375828
[67]	valid_0's l2: 0.282672	valid_0's l1: 0.375679
[68]	valid_0's l2: 0.281948	valid_0's l1: 0.374857
[69]	valid_0's l2: 0.281757	valid_0's l1: 0.374698
[70]	valid_0's l2: 0.281426	valid_0's l1: 0.374544
[71]	valid_0's l2: 0.281674	valid_0's l1: 0.374601
[72]	valid_0's l2: 0.281595	valid_0's l1: 0.374514
[73]	valid_0's l2: 0.281667	valid_0's l1: 0.374465
[74]	valid_0's l2: 0.280996	valid_0's l1: 0.373821
[75]	valid_0's l2: 0.280984	valid_0's l1: 0.373672
[76]	valid_0's l2: 0.28137	valid_0's l1: 0.373763
[77]	valid_0's l2: 0.281331	valid_

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l2: 0.965765	valid_0's l1: 0.792872
[2]	valid_0's l2: 0.818652	valid_0's l1: 0.727385
[3]	valid_0's l2: 0.698199	valid_0's l1: 0.669043
[4]	valid_0's l2: 0.597914	valid_0's l1: 0.617212
[5]	valid_0's l2: 0.51588	valid_0's l1: 0.570794
[6]	valid_0's l2: 0.448705	valid_0's l1: 0.53032
[7]	valid_0's l2: 0.391707	valid_0's l1: 0.493068
[8]	valid_0's l2: 0.345531	valid_0's l1: 0.460549
[9]	valid_0's l2: 0.307197	valid_0's l1: 0.431942
[10]	valid_0's l2: 0.275432	valid_0's l1: 0.406426
[11]	valid_0's l2: 0.247978	valid_0's l1: 0.383029
[12]	valid_0's l2: 0.226594	valid_0's l1: 0.363339
[13]	valid_0's l2: 0.20736	valid_0's l1: 0.34523
[14]	valid_0's l2: 0.191712	valid_0's l1: 0.330008
[15]	valid_0's l2: 0.179381	valid_0's l1: 0.316457
[16]	valid_0's l2: 0.168555	valid_0's l1: 0.304011
[17]	valid_0's l2: 0.159698	valid_0's l1: 0.293508
[18]	valid_0's l2: 0.1515	valid_0's l1: 0.283748
[19]	valid_0's l2: 0.14485	valid_0's l1: 0.

...

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[30]	valid_0's l2: 0.194298	valid_0's l1: 0.332437
[31]	valid_0's l2: 0.19593	valid_0's l1: 0.333722
[32]	valid_0's l2: 0.198875	valid_0's l1: 0.335858
[33]	valid_0's l2: 0.200329	valid_0's l1: 0.336678
[34]	valid_0's l2: 0.201441	valid_0's l1: 0.33775
[35]	valid_0's l2: 0.202059	valid_0's l1: 0.337887
[36]	valid_0's l2: 0.203559	valid_0's l1: 0.33905
[37]	valid_0's l2: 0.204913	valid_0's l1: 0.339836
[38]	valid_0's l2: 0.206213	valid_0's l1: 0.340718
[39]	valid_0's l2: 0.207103	valid_0's l1: 0.341441
[40]	valid_0's l2: 0.208216	valid_0's l1: 0.342466
[41]	valid_0's l2: 0.209047	valid_0's l1: 0.342579
[42]	valid_0's l2: 0.210072	valid_0's l1: 0.343364
[43]	valid_0's l2: 0.210712	valid_0's l1: 0.343614
[44]	valid_0's l2: 0.21129	valid_0's l1: 0.343947
[45]	valid_0's l2: 0.212216	valid_0's l1: 0.344431
[46]	valid_0's l2: 0.212779	valid_0's l1: 0.34464
[47]	valid_0's l2: 0.213081	valid_0's l1: 0.344836
[48]	valid_0's l2: 0.213221	valid_0's l1: 0.344863
[49]	valid_0's l2: 0.213545	valid_0'

[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l2: 1.01771	valid_0's l1: 0.750566
[2]	valid_0's l2: 0.854859	valid_0's l1: 0.680784
[3]	valid_0's l2: 0.718569	valid_0's l1: 0.618232
[4]	valid_0's l2: 0.612447	valid_0's l1: 0.564334
[5]	valid_0's l2: 0.525524	valid_0's l1: 0.517116
[6]	valid_0's l2: 0.451584	valid_0's l1: 0.473396
[7]	valid_0's l2: 0.392585	valid_0's l1: 0.435167
[8]	valid_0's l2: 0.343942	valid_0's l1: 0.402763
[9]	valid_0's l2: 0.302286	valid_0's l1: 0.374069
[10]	valid_0's l2: 0.270615	valid_0's l1: 0.349634
[11]	valid_0's l2: 0.243413	valid_0's l1: 0.328392
[12]	valid_0's l2: 0.221882	valid_0's l1: 0.311373
[13]	valid_0's l2: 0.203177	valid_0's l1: 0.298517
[14]	valid_0's l2: 0.189368	valid_0's l1: 0.288745
[15]	valid_0's l2: 0.177842	valid_0's l1: 0.280194
[16]	valid_0's l2: 0.168554	valid_0's l1: 0.274355
[17]	valid_0's l2: 0.160768	valid_0's l1: 0.269493
[18]	valid_0's l2: 0.153868	valid_0's l1: 0.265766
[19]	valid_0's l2: 0.14872	valid_0's l

C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
C:\Users\PC\anaconda3\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[48]	valid_0's l2: 0.0870571	valid_0's l1: 0.222167
[49]	valid_0's l2: 0.0870211	valid_0's l1: 0.222167
[50]	valid_0's l2: 0.0866158	valid_0's l1: 0.221745
[51]	valid_0's l2: 0.0863363	valid_0's l1: 0.221392
[52]	valid_0's l2: 0.0858906	valid_0's l1: 0.220732
[53]	valid_0's l2: 0.0856914	valid_0's l1: 0.220347
[54]	valid_0's l2: 0.0853209	valid_0's l1: 0.219893
[55]	valid_0's l2: 0.0856083	valid_0's l1: 0.220083
[56]	valid_0's l2: 0.0854171	valid_0's l1: 0.219875
[57]	valid_0's l2: 0.0849945	valid_0's l1: 0.219203
[58]	valid_0's l2: 0.0845533	valid_0's l1: 0.218528
[59]	valid_0's l2: 0.0844014	valid_0's l1: 0.218292
[60]	valid_0's l2: 0.0841445	valid_0's l1: 0.218024
[61]	valid_0's l2: 0.0839386	valid_0's l1: 0.217644
[62]	valid_0's l2: 0.0836089	valid_0's l1: 0.217106
[63]	valid_0's l2: 0.083701	valid_0's l1: 0.216991
[64]	valid_0's l2: 0.0836288	valid_0's l1: 0.216889
[65]	valid_0's l2: 0.0837109	valid_0's l1: 0.21644
[66]	valid_0's l2: 0.0834942	valid_0's l1: 0.216203
[67]	valid_0's

In [133]:
# Convert each dictionary to a DataFrame
LIGHTGBM_spring_df = pd.DataFrame.from_dict(LIGHTGBM_spring_mae, orient='index', columns=['MAE'])
LIGHTGBM_summer_df = pd.DataFrame.from_dict(LIGHTGBM_summer_mae, orient='index', columns=['MAE'])
LIGHTGBM_autumn_df = pd.DataFrame.from_dict(LIGHTGBM_autumn_mae, orient='index', columns=['MAE'])
LIGHTGBM_winter_df = pd.DataFrame.from_dict(LIGHTGBM_winter_mae, orient='index', columns=['MAE'])

LIGHTGBM_mae_df = pd.concat([LIGHTGBM_spring_df, LIGHTGBM_summer_df, LIGHTGBM_autumn_df, LIGHTGBM_winter_df], axis=0)
LIGHTGBM_mae_df.columns = ['LIGHTGBM_MAE']


In [134]:
LIGHTGBM_mae_df

,LIGHTGBM_MAE
114_Spring,2.701167
136_Spring,3.414596
146_Spring,3.128670
177_Spring,1.894243
189_Spring,2.284513
203_Spring,2.003201
232_Spring,1.843246
251_Spring,2.107415
263_Spring,1.796123
276_Spring,2.290810


# KNeighborsRegressor

In [143]:
def KNeighbors_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            x_train_df = pd.DataFrame(x_train)
            x_test_df = pd.DataFrame(x_test)

            # Create a model

            RegModel = KNeighborsRegressor(n_neighbors=10)

            #Creating the model on Training Data
            model=RegModel.fit(x_train_df, y_train)
            y_pred = model.predict(x_test)

            TC_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
            
            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [144]:
KNEIGHBORS_spring_mae, KNEIGHBORS_spring_TC_pred = KNeighbors_model('spring')
KNEIGHBORS_summer_mae, KNEIGHBORS_summer_TC_pred = KNeighbors_model('summer')
KNEIGHBORS_autumn_mae, KNEIGHBORS_autumn_TC_pred = KNeighbors_model('autumn')
KNEIGHBORS_winter_mae, KNEIGHBORS_winter_TC_pred = KNeighbors_model('winter')


In [145]:
# Convert each dictionary to a DataFrame
KNEIGHBORS_spring_df = pd.DataFrame.from_dict(KNEIGHBORS_spring_mae, orient='index', columns=['MAE'])
KNEIGHBORS_summer_df = pd.DataFrame.from_dict(KNEIGHBORS_summer_mae, orient='index', columns=['MAE'])
KNEIGHBORS_autumn_df = pd.DataFrame.from_dict(KNEIGHBORS_autumn_mae, orient='index', columns=['MAE'])
KNEIGHBORS_winter_df = pd.DataFrame.from_dict(KNEIGHBORS_winter_mae, orient='index', columns=['MAE'])


KNEIGHBORS_mae_df = pd.concat([KNEIGHBORS_spring_df, KNEIGHBORS_summer_df, KNEIGHBORS_autumn_df, KNEIGHBORS_winter_df], axis=0)
KNEIGHBORS_mae_df.columns = ['KNEIGHBORS_MAE']


In [146]:
KNEIGHBORS_mae_df

,KNEIGHBORS_MAE
114_Spring,2.805506
136_Spring,3.476859
146_Spring,3.382119
177_Spring,2.204382
189_Spring,2.510596
203_Spring,2.316277
232_Spring,2.203880
251_Spring,2.232903
263_Spring,2.377500
276_Spring,2.486868


In [21]:
MAE_df = pd.concat([linear_mae_df, xgb_mae_df, randomforest_mae_df, MLP_mae_df, ROCKET_mae_df, LIGHTGBM_mae_df, KNEIGHBORS_mae_df, LSTM_mae_df], axis = 1)

type: name 'linear_mae_df' is not defined

In [ ]:
MAE_df.to_csv('modeling_mae.csv', encoding = 'utf-8')

# LSTM

In [18]:
from tensorflow.python.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

def LSTM_model(weather):
    # initialize result dictionaries
    mae_results = {}
    TC_pred_results = {}

    train = pd.read_csv("Train_data.csv")
    train['Date'] = pd.to_datetime(train['Date'])

    # Add filter for months based on weather
    if weather == 'spring':
        train = train[train['Date'].dt.month.isin([2, 3, 4])]
    elif weather == 'summer':
        train = train[train['Date'].dt.month.isin([5, 6, 7])]
    elif weather == 'autumn':
        train = train[train['Date'].dt.month.isin([8, 9, 10])]
    elif weather == 'winter':
        train = train[train['Date'].dt.month.isin([11, 12, 1])]

    # 스케일러 생성
    # 평균 0, 표준편차 1로 스케일링
    scaler = StandardScaler()
    y_scaler = StandardScaler()

    # train 데이터로 스케일러 학습
    x_train = scaler.fit_transform(train.iloc[:,2:12])
    y_train = y_scaler.fit_transform(train.iloc[:,-1].values.reshape(-1, 1))
    
    for file in test_col:
        if weather.capitalize() in file:
            test = pd.read_csv(file, encoding='cp949')
            test = test.iloc[:, :-1]
            test['Date'] = pd.to_datetime(test['Date'])
            
            x_test = scaler.transform(test.iloc[:,2:12])
            y_test = test.iloc[:,-1].values  # add this line

            
            x_train = x_train.reshape((-1, 10, 1))

            # Create a model
            
            # LSTM 모델 생성
            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(10, 1)))  # LSTM 레이어
            model.add(Dense(1))  # 출력 레이어

            # 모델 컴파일
            model.compile(optimizer='adam', loss='mae')

            # Fit the model
            model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=2, validation_split=0.2, callbacks=[early_stopping])

            # Make predictions
            y_pred = model.predict(x_test)
    
            TC_pred = y_scaler.inverse_transform(y_pred)

            # Evaluate the model
            mae = mean_absolute_error(y_test, TC_pred)

            # save the results with the filename as the key
            filename = os.path.splitext(os.path.basename(file))[0]  # extract filename without extension
            mae_results[filename] = mae
            TC_pred_results[filename] = TC_pred

    return mae_results, TC_pred_results

In [19]:
LSTM_spring_mae, LSTM_spring_TC_pred = LSTM_model('spring')
LSTM_summer_mae, LSTM_summer_TC_pred = LSTM_model('summer')
LSTM_autumn_mae, LSTM_autumn_TC_pred = LSTM_model('autumn')
LSTM_winter_mae, LSTM_winter_TC_pred = LSTM_model('winter')


Epoch 1/50
1338/1338 - 4s - loss: 0.3345 - val_loss: 0.2461 - 4s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 4s - loss: 0.3318 - val_loss: 0.2365 - 4s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 4s - loss: 0.3322 - val_loss: 0.2377 - 4s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 5s - loss: 0.3265 - val_loss: 0.2329 - 5s/epoch - 4ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 5s - loss: 0.3373 - val_loss: 0.2544 - 5s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 4s - loss: 0.3331 - val_loss: 0.2403 - 4s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 4s - loss: 0.3508 - val_loss: 0.2551 - 4s/epoch - 3ms/step
70/70 [==============================] - 0s 1ms/step
Epoch 1/50
1338/1338 - 4s - loss: 0.3322 - val_loss: 0.

In [20]:
# Convert each dictionary to a DataFrame
LSTM_spring_df = pd.DataFrame.from_dict(LSTM_spring_mae, orient='index', columns=['MAE'])
LSTM_summer_df = pd.DataFrame.from_dict(LSTM_summer_mae, orient='index', columns=['MAE'])
LSTM_autumn_df = pd.DataFrame.from_dict(LSTM_autumn_mae, orient='index', columns=['MAE'])
LSTM_winter_df = pd.DataFrame.from_dict(LSTM_winter_mae, orient='index', columns=['MAE'])


LSTM_mae_df = pd.concat([LSTM_spring_df, LSTM_summer_df, LSTM_autumn_df, LSTM_winter_df], axis=0)
LSTM_mae_df.columns = ['LSTM_MAE']



In [23]:
LSTM_mae_df.to_csv("LSTM.csv", encoding = 'utf-8')